In [1]:
import os
import numpy as np
from tqdm import tqdm_notebook
import pickle as pkl
import jieba

In [2]:
UNK, PAD = '<UNK>', '<PAD>'  # 未知字，padding符号

In [3]:
def build_vocab(file_path, tokenizer, min_freq):
    vocab_dic = {}
    with open(file_path, 'r', encoding='UTF-8') as f:
        for line in tqdm_notebook(f):
            lin = line.strip()
            if not lin:
                continue
            content = lin.split('\t')[2]
            #增加分词步骤
            seg_list = jieba.cut(content)  
            content = " ".join(seg_list)
            for word in tokenizer(content):
                vocab_dic[word] = vocab_dic.get(word, 0) + 1
        vocab_list = sorted([_ for _ in vocab_dic.items() if _[1] >= min_freq], key=lambda x: x[1], reverse=True)
        vocab_dic = {word_count[0]: idx for idx, word_count in enumerate(vocab_list)}
        vocab_dic.update({UNK: len(vocab_dic), PAD: len(vocab_dic) + 1})
    return vocab_dic

In [4]:
train_dir = "zhengshi3/zhengshi_infouse_trn"
vocab_dir = "zhengshi3/vocab.pkl"
if os.path.exists(vocab_dir):
    word_to_id = pkl.load(open(vocab_dir, 'rb'))
else:
    tokenizer = lambda x: x.split(' ')  #以词为单位构建词表(数据集中词之间以空格隔开)
    #tokenizer = lambda x: [y for y in x]  #以字为单位构建词表
    word_to_id = build_vocab(train_dir, tokenizer=tokenizer, min_freq=1)
    pkl.dump(word_to_id, open(vocab_dir, 'wb'))

In [5]:
emb_dim = 200
pretrain_dir = "../paper/cloudcode/info_classification/data/Tencent_AILab_ChineseEmbedding.txt"
filename_trimmed_dir = "../paper/cloudcode/info_classification/data/Tencent_AILab_ChineseEmbedding"
embeddings = np.random.rand(len(word_to_id), emb_dim)
with open(pretrain_dir, 'r', encoding='utf-8') as f:
    for line in tqdm_notebook(f):
        if len(line) != 0:
            l = line.strip().split(' ')
            word = l[0]
            embedding = l[1:]
            if word in word_to_id:
                word_idx = word_to_id[word]
                embeddings[word_idx] = np.asarray(embedding, dtype='float32')

np.savez_compressed(filename_trimmed_dir, embeddings=embeddings)